In [19]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
qr = QuantumRegister(2, name='qr')
cr = ClassicalRegister(2, name='cr')
qc = QuantumCircuit(qr, cr, name='qc')
# TIMESTAMP: 1738173780.410562

# Apply gate operations
# <START_GATES>
# qc.ccx(qr[5], qr[2], qr[6])
qc.ecr(qr[0], qr[1])
# qc.cp(1.563586, qr[4], qr[6])
# qc.cry(4.092326, qr[10], qr[4])
# qc.t(qr[0])
# <END_GATES>

# Section: Measurement
qc.measure(qr, cr)

In [20]:
from pytket.extensions.qiskit import qiskit_to_tk

tkqc = qiskit_to_tk(qc)

In [21]:
from pytket.qasm import circuit_to_qasm_str
print(circuit_to_qasm_str(tkqc))

OPENQASM 2.0;
include "qelib1.inc";

gate ecr ecrq0,ecrq1 {
u3(1.0*pi,1.5*pi,1.0*pi) ecrq0;
u3(3.5*pi,1.5*pi,0.5*pi) ecrq1;
cx ecrq0,ecrq1;
}
qreg qr[2];
creg cr[2];
ecr qr[0],qr[1];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];



In [23]:
from pytket.circuit.display import render_circuit_jupyter
from pytket.qasm import circuit_to_qasm_str
from pytket.passes import (
    FullPeepholeOptimise, PeepholeOptimise2Q, RemoveRedundancies,
    EulerAngleReduction, KAKDecomposition,
    CliffordPushThroughMeasures, FlattenRegisters,
    PauliSimp, GreedyPauliSimp,
    OptimisePhaseGadgets,
    ZXGraphlikeOptimisation
)
from pytket.circuit import OpType
from copy import deepcopy

optimization_passes = {
    # "FullPeepholeOptimise": FullPeepholeOptimise(),
    # "PeepholeOptimise2Q": PeepholeOptimise2Q(),
    # "RemoveRedundancies": RemoveRedundancies(),
    # "EulerAngleReduction": EulerAngleReduction(q=OpType.Rz, p=OpType.Rx),
    # "KAKDecomposition": KAKDecomposition(),
    "CliffordPushThroughMeasures": CliffordPushThroughMeasures(),
    #     "FlattenRegisters": FlattenRegisters(),
    #     "PauliSimp": PauliSimp(),
    #     "GreedyPauliSimp": GreedyPauliSimp(),
    #     "OptimisePhaseGadgets": OptimisePhaseGadgets(),
    #     "ZXGraphlikeOptimisation": ZXGraphlikeOptimisation()
}

for opt_pass_name, optimization_pass in optimization_passes.items():
    print(f"Applying {opt_pass_name}")
    i_qc = deepcopy(tkqc)
    optimization_pass.apply(i_qc)
    render_circuit_jupyter(i_qc)
    i_opt_circuit_qasm = circuit_to_qasm_str(
        i_qc, header="hqslib1", maxwidth=200)
    # i_qc, header="qelib1", maxwidth=200)

Applying CliffordPushThroughMeasures


In [25]:
print(i_opt_circuit_qasm)

OPENQASM 2.0;
include "hqslib1.inc";

qreg qr[2];
creg cr[2];
creg permutation_scratch[3];
measure qr[0] -> cr[0];
measure qr[1] -> cr[1];
permutation_scratch[2] = 1;
permutation_scratch[0] = cr[0] ^ permutation_scratch[0];
permutation_scratch[1] = cr[1] ^ permutation_scratch[1];
cr[0] = permutation_scratch[0];
cr[1] = permutation_scratch[1];

